In [48]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
#1.下载mnist数据集
mnist = input_data.read_data_sets('MNIST_data',one_hot=True) #让label转换成one-hot格式

#2对数据进行操作
#2.1分成多个batch
batch_size = 100
n_batch = mnist.train.num_examples // batch_size  #//整除

#2.2为data 和 label 创建plcaeholder   #！None 具体多少数量之后指定
data = tf.placeholder(tf.float32,[None,784])
label = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [49]:
#3.创建神经网络
#3.1权重参数 和 偏置项
W1 = tf.Variable(tf.truncated_normal([784,580],stddev=0.1))
b1 = tf.Variable(tf.zeros([580])+0.1)
L1 = tf.nn.relu(tf.matmul(data,W1)+b1)
#drop out
L1_drop = tf.nn.dropout(L1,keep_prob)

#### 对权重参数进行初始化  对偏置项进行初始化

In [50]:
#第二层
W2 = tf.Variable(tf.truncated_normal([580,580],stddev=0.1))
b2 = tf.Variable(tf.zeros([580])+0.1)
L2 =  tf.nn.relu(tf.matmul(L1_drop,W2)+b2)
L2_drop = tf.nn.dropout(L2,keep_prob)
#第三层
W3 = tf.Variable(tf.truncated_normal([580,340],stddev=0.1))
b3 = tf.Variable(tf.zeros([340])+0.1)
L3 = tf.nn.relu(tf.matmul(L1_drop,W3)+b3)
L3_drop = tf.nn.dropout(L3,keep_prob)
#输出层
W4 = tf.Variable(tf.truncated_normal([340,10],stddev=0.1))
b4 = tf.Variable(tf.zeros([10])+0.1)

In [51]:
#3.2softmax 为分类求出概率
prediction = tf.nn.softmax(tf.matmul(L3_drop,W4)+b4)   
#3.3使用二次代价函数计算loss值
# loss = tf.reduce_mean(tf.square(prediction-label))
#使用交叉熵 计算loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=label,logits=prediction))

#### 非线性使用交叉熵加上softmax

In [52]:
#3.4使用梯度下降法
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

#4.求出准确率
#4.1 得出一个判断正确与否的 布尔类型list
#! arg_max（） 返回一维张量中最大的索引值 1表示按行比较   这里 前一个就是取出one-hot的那个1所在的位置 后一个就是取出概率值最大的位置
correct_prediction = tf.equal(tf.argmax(label,1),tf.argmax(prediction,1))
#4.2 计算正确率   将布尔型列表转换成 0 1 求出平均值就是正确率
accuracy = (tf.reduce_mean(tf.cast(correct_prediction,tf.float32)))

#5. session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(31):
        for batch in range(n_batch):
            #每次向下取出一个batch的数据
            train_data,train_label = mnist.train.next_batch(batch_size) #？为什么这里mnist 有 train方法...
            sess.run(train_step,feed_dict={data:train_data,label:train_label,keep_prob:1.0})
            #传入测试集的数据和标签
        test_acc = sess.run(accuracy,feed_dict={data:mnist.test.images,label:mnist.test.labels,keep_prob:1.0})
        train_acc = sess.run(accuracy,feed_dict={data:mnist.train.images,label:mnist.train.labels,keep_prob:1.0})
        print(epoch,"train:",train_acc,"test:",test_acc)

0 train: 0.95443636 test: 0.9511
1 train: 0.97296363 test: 0.9653
2 train: 0.9761636 test: 0.9681
3 train: 0.97914547 test: 0.9729
4 train: 0.9808 test: 0.9705
5 train: 0.9815636 test: 0.974
6 train: 0.9855818 test: 0.9771
7 train: 0.98843634 test: 0.9783
8 train: 0.98801816 test: 0.9785
9 train: 0.98803633 test: 0.9759
10 train: 0.98701817 test: 0.9705
11 train: 0.98803633 test: 0.9758
12 train: 0.9885455 test: 0.9762
13 train: 0.98892725 test: 0.9793
14 train: 0.9908 test: 0.9783
15 train: 0.9908182 test: 0.9782
16 train: 0.9892727 test: 0.978
17 train: 0.9919818 test: 0.9804
18 train: 0.9896 test: 0.9773
19 train: 0.9919091 test: 0.9801
20 train: 0.992 test: 0.979
21 train: 0.9905818 test: 0.9779
22 train: 0.99114543 test: 0.9764
23 train: 0.9911636 test: 0.9775
24 train: 0.9915818 test: 0.9776
25 train: 0.99154544 test: 0.9788
26 train: 0.9924 test: 0.9775
27 train: 0.9909818 test: 0.9769
28 train: 0.9903273 test: 0.9758
29 train: 0.9921091 test: 0.9794
30 train: 0.9927818 test: 0.

In [ ]:
1.1神经元不是越多越好 要合适 神经元超过一定限度 收敛应该是越慢
1.2如果没有超过限度 神经元增长的话  收敛会加快
2.目前来说学习率越低 收敛越慢  而且初始准确率比较低
3.随机梯度下降法 对于梯度下降来说 收敛加快  （好像不是很稳定）

In [ ]:
relu  drop-out 1  学习率0.2   神经元比较少  340 280
0 train: 0.91752726 test: 0.9202
1 train: 0.9434909 test: 0.9401
2 train: 0.95158184 test: 0.9466
3 train: 0.95314544 test: 0.9485
4 train: 0.9642364 test: 0.9557
5 train: 0.96590906 test: 0.9585
6 train: 0.9694909 test: 0.9601
7 train: 0.9734727 test: 0.9634
8 train: 0.97587276 test: 0.9643
9 train: 0.97810906 test: 0.9672
10 train: 0.97925454 test: 0.9679
11 train: 0.9805091 test: 0.9673
12 train: 0.9810727 test: 0.9668
13 train: 0.9829818 test: 0.9698
14 train: 0.9839091 test: 0.9702
15 train: 0.9844364 test: 0.9703
16 train: 0.9849455 test: 0.9702
17 train: 0.9860909 test: 0.9711
18 train: 0.9866727 test: 0.9722
19 train: 0.987 test: 0.9724
20 train: 0.9876 test: 0.9721
21 train: 0.98807275 test: 0.9714
22 train: 0.9882182 test: 0.972
23 train: 0.98865455 test: 0.9731
24 train: 0.98890907 test: 0.9726
25 train: 0.9892727 test: 0.9729
26 train: 0.98932725 test: 0.9734
27 train: 0.98956364 test: 0.9728
28 train: 0.9900182 test: 0.974
29 train: 0.9901636 test: 0.9738
30 train: 0.9902727 test: 0.9736

In [ ]:
relu  drop-out 1  学习率0.1 神经元比较少  340 280
0 train: 0.89974546 test: 0.9038
1 train: 0.9243818 test: 0.9278
2 train: 0.9341818 test: 0.934
3 train: 0.9438 test: 0.94
4 train: 0.94865453 test: 0.9456
5 train: 0.95181817 test: 0.9459
6 train: 0.9558909 test: 0.951
7 train: 0.959 test: 0.9518
8 train: 0.9631636 test: 0.9554
9 train: 0.9649818 test: 0.9559
10 train: 0.9668546 test: 0.9569
11 train: 0.96843636 test: 0.9581
12 train: 0.97098184 test: 0.9599
13 train: 0.9724909 test: 0.9627
14 train: 0.97345454 test: 0.964
15 train: 0.9738 test: 0.9636
16 train: 0.9759273 test: 0.9651
17 train: 0.9769818 test: 0.9659
18 train: 0.97816366 test: 0.9666
19 train: 0.979 test: 0.9674
20 train: 0.97974545 test: 0.9669
21 train: 0.9806 test: 0.9682
22 train: 0.98145455 test: 0.9673
23 train: 0.98205453 test: 0.9677
24 train: 0.98254544 test: 0.9694
25 train: 0.98276365 test: 0.9684
26 train: 0.9833091 test: 0.9695
27 train: 0.9836182 test: 0.9694
28 train: 0.9841818 test: 0.9697
29 train: 0.9843818 test: 0.9693
30 train: 0.9849091 test: 0.9691


In [ ]:
relu  drop-out 1  学习率0.2  580  340

0 train: 0.9283636 test: 0.9325
1 train: 0.9404727 test: 0.9362
2 train: 0.95567274 test: 0.9506
3 train: 0.95916367 test: 0.9526
4 train: 0.96983635 test: 0.9604
5 train: 0.97056365 test: 0.9608
6 train: 0.9742727 test: 0.9632
7 train: 0.9783091 test: 0.9672
8 train: 0.9802182 test: 0.9685
9 train: 0.9816727 test: 0.9687
10 train: 0.98203635 test: 0.9683
11 train: 0.9825818 test: 0.9683
12 train: 0.9856182 test: 0.9724
13 train: 0.9864182 test: 0.9717
14 train: 0.9869818 test: 0.9731
15 train: 0.9873818 test: 0.9732
16 train: 0.98841816 test: 0.9736
17 train: 0.9881091 test: 0.9732
18 train: 0.9887273 test: 0.9737
19 train: 0.9893636 test: 0.974
20 train: 0.98972726 test: 0.9749
21 train: 0.9901091 test: 0.9751
22 train: 0.99047273 test: 0.9749
23 train: 0.99056363 test: 0.9764
24 train: 0.9906909 test: 0.9758
25 train: 0.99094546 test: 0.9755
26 train: 0.9911818 test: 0.9756
27 train: 0.9913273 test: 0.9753
28 train: 0.9914727 test: 0.9748
29 train: 0.99163634 test: 0.9753
30 train: 0.9917273 test: 0.9759

In [ ]:
relu  drop-out 1  学习率0.1  580  340
0 train: 0.8213818 test: 0.8246
1 train: 0.84816366 test: 0.8479
2 train: 0.8587273 test: 0.8574
3 train: 0.8629091 test: 0.8607
4 train: 0.9488364 test: 0.946
5 train: 0.9567636 test: 0.9529
6 train: 0.9606 test: 0.9552
7 train: 0.96485454 test: 0.9578
8 train: 0.9663091 test: 0.9579
9 train: 0.9705091 test: 0.9604
10 train: 0.9725636 test: 0.9625
11 train: 0.97476363 test: 0.9643
12 train: 0.9763091 test: 0.964
13 train: 0.97727275 test: 0.9659
14 train: 0.97932726 test: 0.9686
15 train: 0.97947276 test: 0.9689
16 train: 0.9814364 test: 0.9697
17 train: 0.9817273 test: 0.9692
18 train: 0.98252726 test: 0.9702
19 train: 0.9834909 test: 0.9701
20 train: 0.984 test: 0.9715
21 train: 0.9843636 test: 0.9716
22 train: 0.98472726 test: 0.9704
23 train: 0.9856 test: 0.9725
24 train: 0.98603636 test: 0.9723
25 train: 0.9864182 test: 0.9722
26 train: 0.98678184 test: 0.9732
27 train: 0.98718184 test: 0.9731
28 train: 0.9873273 test: 0.9733
29 train: 0.9877091 test: 0.9736
30 train: 0.98807275 test: 0.9739

In [ ]:
relu  drop-out 1  随机梯度下降法 学习率0.1  580  340  
0 train: 0.95443636 test: 0.9511
1 train: 0.97296363 test: 0.9653
2 train: 0.9761636 test: 0.9681
3 train: 0.97914547 test: 0.9729
4 train: 0.9808 test: 0.9705
5 train: 0.9815636 test: 0.974
6 train: 0.9855818 test: 0.9771
7 train: 0.98843634 test: 0.9783
8 train: 0.98801816 test: 0.9785
9 train: 0.98803633 test: 0.9759
10 train: 0.98701817 test: 0.9705
11 train: 0.98803633 test: 0.9758
12 train: 0.9885455 test: 0.9762
13 train: 0.98892725 test: 0.9793
14 train: 0.9908 test: 0.9783
15 train: 0.9908182 test: 0.9782
16 train: 0.9892727 test: 0.978
17 train: 0.9919818 test: 0.9804
18 train: 0.9896 test: 0.9773
19 train: 0.9919091 test: 0.9801
20 train: 0.992 test: 0.979
21 train: 0.9905818 test: 0.9779
22 train: 0.99114543 test: 0.9764
23 train: 0.9911636 test: 0.9775
24 train: 0.9915818 test: 0.9776
25 train: 0.99154544 test: 0.9788
26 train: 0.9924 test: 0.9775
27 train: 0.9909818 test: 0.9769
28 train: 0.9903273 test: 0.9758
29 train: 0.9921091 test: 0.9794
30 train: 0.9927818 test: 0.9809